<a href="https://colab.research.google.com/github/elekzordd/Test/blob/main/20250120_GeoSpatial_data_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### 0. Install all necessary packages
!pip install gdal
!pip install duckdb
!pip  install geopandas
!pip install pyarrow

In [7]:
### 1. Imports

#1.a. Import the `duckdb` library for querying and managing data.
import duckdb

#1.b. Import the `geopandas` library for working with geospatial data.
import geopandas

#1.c. Import the `pyarrow` library for handling data in Apache Arrow format.
import pyarrow

In [9]:
%reload_ext watermark
%watermark -a "KURWISKOOOOO"

Author: KURWISKOOOOO



In [10]:
### 2. Installing DuckDB extensions

#2.a. Install the `httpfs` extension for handling HTTP file systems.
duckdb.sql('INSTALL httpfs')

#2.b. Load the `httpfs` extension.
duckdb.sql('LOAD httpfs')

#2.c. Force install the `spatial` extension from the specified URL.
duckdb.sql("FORCE INSTALL spatial FROM 'http://nightly-extensions.duckdb.org';")

#2.d. Load the `spatial` extension for geospatial data processing.
duckdb.sql('LOAD spatial')

In [11]:
### 3. Initializing DuckDB for in-memory processing

#3.a. Create a DuckDB database instance with in-memory storage.
con = duckdb.connect(database=":memory:")

In [12]:
### 4. Installing and loading extensions (an alternative to what was shown earlier)

#4.a. Install and load the `httpfs` extension.
con.install_extension('httpfs')
con.load_extension('httpfs')

#4.b. Install and load the `spatial` extension.
con.install_extension('spatial')
con.load_extension('spatial')

In [13]:
### 5. S3 file address in the AWS cloud
prefix = "s3://us-west-2.opendata.source.coop/vida/google-microsoft-open-buildings/geoparquet"

In [14]:
### 6. Partition type
partitions = "by_country"

In [15]:
### 7. Country ISO code
country_iso = "LTU"

In [16]:
### 8. Creating a table with building data for Lithuania

%%time

con.execute(f"""
    CREATE OR REPLACE TABLE lithuania_buildings AS
      SELECT * FROM parquet_scan('{prefix}/by_country_s2/country_iso={country_iso}/*.parquet')
""")



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 5.47 s, sys: 1.72 s, total: 7.19 s
Wall time: 30.1 s


In [18]:
### 9. Table summary

%%time

con.query('DESCRIBE lithuania_buildings')

CPU times: user 2.3 ms, sys: 819 µs, total: 3.11 ms
Wall time: 4.78 ms


┌───────────────────┬────────────────────────────────────────────────────────────┬─────────┬─────────┬─────────┬─────────┐
│    column_name    │                        column_type                         │  null   │   key   │ default │  extra  │
│      varchar      │                          varchar                           │ varchar │ varchar │ varchar │ varchar │
├───────────────────┼────────────────────────────────────────────────────────────┼─────────┼─────────┼─────────┼─────────┤
│ boundary_id       │ BIGINT                                                     │ YES     │ NULL    │ NULL    │ NULL    │
│ bf_source         │ VARCHAR                                                    │ YES     │ NULL    │ NULL    │ NULL    │
│ confidence        │ DOUBLE                                                     │ YES     │ NULL    │ NULL    │ NULL    │
│ area_in_meters    │ DOUBLE                                                     │ YES     │ NULL    │ NULL    │ NULL    │
│ s2_id         

In [19]:
### 10. Record count

%%time

con.query('SELECT COUNT(*) FROM lithuania_buildings;')

CPU times: user 2.66 ms, sys: 0 ns, total: 2.66 ms
Wall time: 1.97 ms


┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      2160393 │
└──────────────┘

In [20]:
### 11. Selecting the source and count

%%time

con.query('SELECT bf_source AS data_source, COUNT(*) AS count FROM lithuania_buildings GROUP BY data_source;')

CPU times: user 3.28 ms, sys: 0 ns, total: 3.28 ms
Wall time: 8.98 ms


┌─────────────┬─────────┐
│ data_source │  count  │
│   varchar   │  int64  │
├─────────────┼─────────┤
│ microsoft   │ 2160393 │
└─────────────┴─────────┘

In [24]:
### 12. Using SELECT is not mandatory with DuckDB

%%time

# con.query('''FROM lithuania_buildings WHERE bf_source = 'microsoft' LIMIT 10;''')

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.34 µs


In [25]:
### 13. We can work with data from other countries

prefix = "s3://us-west-2.opendata.source.coop/vida/google-microsoft-open-buildings/geoparquet"
partitions = "by_country_s2"
country_iso = "POL"

In [26]:
### 14. Define an another SQL query

query1 = f"""
    CREATE TABLE pol_buildings AS
    SELECT s2_id, COUNT(geometry) AS buildings_count
    FROM parquet_scan('{prefix}/{partitions}/country_iso={country_iso}/*.parquet')
    GROUP BY(s2_id)
"""

In [27]:
### 15. Execute the query

%%time

duckdb.sql(query1)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 15.4 s, sys: 2.46 s, total: 17.9 s
Wall time: 3min 19s


In [28]:
### 16. Execute query

%%time

duckdb.sql("SELECT * FROM pol_buildings").show()

┌─────────────────────┬─────────────────┐
│        s2_id        │ buildings_count │
│        int64        │      int64      │
├─────────────────────┼─────────────────┤
│ 4899916394579099648 │        17889682 │
└─────────────────────┴─────────────────┘

CPU times: user 4.75 ms, sys: 0 ns, total: 4.75 ms
Wall time: 4.86 ms


In [29]:
### 17. Define the SQL query

query2 = f"""
    SELECT ROUND(AVG(buildings_count), 0) AS avg_num_buildings
    FROM pol_buildings
"""

In [30]:
### 18. Execute the query and display the result

%%time

duckdb.sql(query2).show()

┌───────────────────┐
│ avg_num_buildings │
│      double       │
├───────────────────┤
│        17889682.0 │
└───────────────────┘

CPU times: user 8.06 ms, sys: 0 ns, total: 8.06 ms
Wall time: 18.2 ms


In [31]:
### 19. Partition type and country ISO code

partitions = "by_country_s2"
country_iso = "ERI"

In [32]:
### 20. Creating or replacing the table for Lesotho buildings

%%time

duckdb.sql(f"""
    CREATE OR REPLACE TABLE eri_buildings AS
    SELECT *
    FROM parquet_scan('{prefix}/{partitions}/country_iso={country_iso}/*.parquet')
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 3.72 s, sys: 1.02 s, total: 4.74 s
Wall time: 20.2 s


In [33]:
### 21. Execute the query

%%time

duckdb.sql(f"""
    SELECT bf_source, COUNT(*) AS buildings_count
    FROM eri_buildings
    GROUP BY bf_source;
""").show()

┌───────────┬─────────────────┐
│ bf_source │ buildings_count │
│  varchar  │      int64      │
├───────────┼─────────────────┤
│ google    │         1511456 │
│ microsoft │           44687 │
└───────────┴─────────────────┘

CPU times: user 124 ms, sys: 0 ns, total: 124 ms
Wall time: 67.2 ms


In [34]:
### 22. Data source from Google Research

prefix = "s3://us-west-2.opendata.source.coop/google-research-open-buildings/geoparquet-by-country"

In [35]:
### 23. Country code

country = "ER"

In [38]:
### 24. Creating or replacing the table for Lesotho buildings from Google dataset

%%time

duckdb.sql(f"""
    CREATE OR REPLACE TABLE eri_buildings_google AS
    SELECT *
    FROM '{prefix}/country_iso={country}/{country}.parquet'
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 2.49 s, sys: 549 ms, total: 3.03 s
Wall time: 4.65 s


In [39]:
### 25. Execute the query to count records in the Google dataset for Lesotho

%%time

duckdb.sql("SELECT COUNT(*) FROM eri_buildings_google").show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      1511002 │
└──────────────┘

CPU times: user 5.31 ms, sys: 0 ns, total: 5.31 ms
Wall time: 4.24 ms


In [40]:
### 27. File with the Area of Interest (AOI)

file = "boundaries.geojson"

In [48]:
### 28. Execute the query to create a table for the Area of Interest (AOI)

%%time

duckdb.sql(f"""
    CREATE OR REPLACE TABLE aoi AS
    SELECT *
    FROM ST_Read('{file}')
""")

CPU times: user 3.96 ms, sys: 775 µs, total: 4.74 ms
Wall time: 4.77 ms


In [49]:
### 29. Execute the query to display the Area of Interest (AOI) table

%%time

duckdb.sql("SELECT * FROM aoi").show()

┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                                                                                                      geom                                                                                                                       │
│                                                                                                                    geometry                                                                                                                     │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ POLYGON ((38.944048967

In [56]:
### 30. Define the SQL query to clip Erithrea buildings with the Area of Interest (AOI)

query3 = """
CREATE TABLE eri_buildings_clipped AS
SELECT ST_Intersection(b.geometry, a.geom) AS geom, b.bf_source
FROM eri_buildings b, aoi a
WHERE ST_Intersects(b.geometry, a.geom);
"""

In [57]:
### 31. Execute the query to create the clipped table for Erithrea buildings

%%time

duckdb.sql(query3)

CPU times: user 1.35 s, sys: 3.89 ms, total: 1.36 s
Wall time: 731 ms


In [58]:
#32. Execute the query to count records in the clipped table for Erithrea buildings

%%time

duckdb.sql("SELECT COUNT(*) FROM eri_buildings_clipped").show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        22598 │
└──────────────┘

CPU times: user 4.39 ms, sys: 45 µs, total: 4.43 ms
Wall time: 3.03 ms


In [59]:
### 33. Define the SQL query to clip Google dataset buildings with the Area of Interest (AOI)

query4 = """
CREATE TABLE eri_buildings_google_clipped AS
SELECT ST_Intersection(b.geometry, a.geom) AS geom
FROM eri_buildings_google b, aoi a
WHERE ST_Intersects(b.geometry, a.geom);
"""

In [60]:
### 34. Execute the query to create the clipped table for Google dataset buildings

%%time

duckdb.sql(query4)

CPU times: user 1.03 s, sys: 8.66 ms, total: 1.04 s
Wall time: 568 ms


In [62]:
### 35. Execute the query to count records in the clipped table for Google dataset buildings

%%time

duckdb.sql("SELECT COUNT(*) FROM eri_buildings_google_clipped").show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        22478 │
└──────────────┘

CPU times: user 3.91 ms, sys: 0 ns, total: 3.91 ms
Wall time: 3.92 ms


In [64]:
### 36. Define the SQL query to find buildings without intersection

query5 = """
CREATE TABLE eri_no_intersection AS
SELECT m.*
FROM eri_buildings_clipped m
WHERE NOT EXISTS (
    SELECT 1
    FROM eri_buildings_google_clipped g
    WHERE ST_Intersects(m.geom, g.geom)
);
"""

In [65]:
### 37. Execute the query to create the table of non-intersecting buildings

%%time

duckdb.sql(query5)

CPU times: user 910 ms, sys: 23.9 ms, total: 934 ms
Wall time: 931 ms


In [66]:
### 38. Execute the query to count records in the non-intersecting buildings table

%%time

duckdb.sql("SELECT count(*) FROM eri_no_intersection").show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│          121 │
└──────────────┘

CPU times: user 3.18 ms, sys: 21 µs, total: 3.2 ms
Wall time: 3.18 ms


In [67]:
### 39. Export the clipped Lesotho buildings dataset to FlatGeobuf format

%%time

output_file = "dataset1.fgb"
duckdb.sql(f"""
    COPY (SELECT * FROM eri_buildings_clipped)
    TO '{output_file}'
    WITH (FORMAT GDAL, DRIVER 'FlatGeobuf');
""")

CPU times: user 348 ms, sys: 27.3 ms, total: 376 ms
Wall time: 227 ms


In [87]:
duckdb.sql(f"""
SELECT *, ST_AsWKB(ST_Envelope(geometry)) AS geometry
        FROM eri_buildings
""").show()

┌─────────────┬───────────┬────────────┬────────────────┬─────────────────────┬─────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────┬───────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ boundary_id │ bf_source │ confidence │ area_in_meters │        s2_id        │ country_iso │                                                                                                                               geometry                                                                   

In [ ]:
# Source:
# https://towardsdatascience.com/handling-billions-of-records-in-minutes-with-sql-%EF%B8%8F-484d2d6027bc#bypass